# Test tf behaviour

This notebook is to test the function of the code snippets from the research paper and make sure that I am recoding to pytorch properly

In [5]:
import tensorflow as tf

#### shape_list

In [3]:
from tf_code.self_atten_utils_tf import shape_list

In [6]:
exp_tensor = tf.random.normal(
    shape = [10, 3, 60, 60]
)

In [7]:
exp_tensor.shape

TensorShape([10, 3, 60, 60])

In [12]:
type(exp_tensor)

tensorflow.python.framework.ops.EagerTensor

In [9]:
result = shape_list(exp_tensor)
result

[10, 3, 60, 60]

In [11]:
type(result)

list

#### split_heads_2d

In [13]:
from tf_code.self_atten_utils_tf import split_heads_2d

In [14]:
result_2 = split_heads_2d(exp_tensor, 2)
result_2.shape

TensorShape([10, 2, 3, 60, 30])

###### explore workflow of split_heads

image -> conv -> conv -> split -> split_heads